In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, confusion_matrix, ConfusionMatrixDisplay, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.cluster import DBSCAN

import seaborn as sns
import time

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/online-payments-fraud-detection-dataset/PS_20174392719_1491204439457_log.csv


In [2]:
df = pd.read_csv("/kaggle/input/online-payments-fraud-detection-dataset/PS_20174392719_1491204439457_log.csv")
df_2 = pd.read_csv("/kaggle/input/online-payments-fraud-detection-dataset/PS_20174392719_1491204439457_log.csv")

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
df.info() # Sütunları ve veri tiplerini gösterir

In [ ]:
df.head(10) # Veri setindeki ilk 10 satırı gösterir

In [ ]:
df.isnull().sum() # Sütunlardaki boş (null) değer sayısını gösterir. 
# Eğer boş değer varsa gerekli yöntemlerle doldurulmalıdır. Boş değer yoksa sıkıntı yoktur.

In [ ]:
df.describe().apply(lambda x: x.apply('{0:.2f}'.format)) #Sütunlarla ilgili çeşitli sayısal bilgiler veriyor
# df.describe() yapınca tüm bilgiler bilimsel gösterim şeklinde verildiğinden onları floata çevirdim

In [ ]:
df[df['isFraud'] == 1]['amount'].describe().apply(lambda x: '{0:.2f}'.format(x)) # Fraud işlemlerde amount hakkında genel bilgi verir 

In [ ]:
#Fraud ve fraud olmayan işlemlerin yüzdelerini pie chart ile gösterir

total = len(df['isFraud'])
count_0 = df['isFraud'].value_counts()[0]
count_1 = df['isFraud'].value_counts()[1]
print(count_1)

per_count_0 = (count_0 / total) * 100
per_count_1 = (count_1 / total) * 100

labels = ['Not Fraud', 'Fraud']
sizes = [per_count_0, per_count_1]
colors = ['green', 'red']

fig, ax = plt.subplots()
ax.pie(sizes, labels = labels, colors = colors, autopct='%1.1f%%')
plt.title('Fraud vs Not Fraud Transactions')
plt.show()
print('Percentage of Frauds:', per_count_1)
print('Percentage of Not Frauds:', per_count_0)

In [ ]:
# Fraud işlemlerin fiyat aralıklarına göre dağılımının bar chart ile gösterimi

bins = [0, 100000, 500000, 1000000, float('inf')]
labels = ['0-100K', '100K-500K', '500K-1M', '1M+']

# Fraud işlemleri için 'amount' sütununu doğrudan gruplandırma yapılması
fraud_df = df[df['isFraud'] == 1]

# Yeni sütun oluşturmadan pd.cut ile gruplandırma ve sayım işlemi
fraud_count_by_amount_range = pd.cut(fraud_df['amount'], bins=bins, labels=labels, right=False).value_counts().sort_index()

plt.figure(figsize=(8, 6))
ax = fraud_count_by_amount_range.plot(kind='bar', color='red', edgecolor='black')

for p in ax.patches:
    ax.text(p.get_x() + p.get_width() / 2, 
            p.get_height() + 1,
            int(p.get_height()),  # sayının integer formatında gösterilmesi
            ha='center', va='bottom')

plt.title('Fraud Transactions Count by Amount Range')
plt.xlabel('Amount Range')
plt.ylabel('Number of Fraud Transactions')
plt.xticks(rotation=45)
plt.show()


In [ ]:
#isFlaggedFraud sütunundaki tahminlerin doğruluğunun oranınım pie chart ile gösterimi

trueFlag = ((df['isFraud'] == df['isFlaggedFraud'])).sum()

falseFlag = ((df['isFraud'] != df['isFlaggedFraud'])).sum()

totalDatas = len(df['isFraud'])
total = trueFlag + falseFlag

true_percentage = round((trueFlag / totalDatas) * 100, 2)
false_percentage = round((falseFlag / totalDatas) * 100, 2)

labels = ['True Predictions', 'False Predictions']
sizes = [true_percentage, false_percentage]
colors = ['green', 'red']
explode = (0, 0.1)

fig, ax = plt.subplots()
ax.pie(sizes, explode=explode, labels=labels, colors = colors, autopct='%1.2f%%',
       shadow=True, startangle=90)
plt.title('True vs False Predictions')
plt.show()

In [ ]:
let = LabelEncoder() #Numerik olmayan veriler öğrenme algoritmaları için sorun yaratabilir
#String veriler var ve bunları numeriğe çevirmek için label encoder fonksiyonu çağrılır.

In [ ]:
df['type'] = let.fit_transform(df['type']) #Veriler numeriğe dönüştürülür
category_mapping = dict(zip(let.classes_, let.transform(let.classes_))) #Dictionary veri yapısı kullanılarak hangi stringin hangi numerik değer aldığı gösterilir.
print(category_mapping)

In [ ]:
# Karmaşıklığı azaltıp algoritmanın performansı için 

#Denetimsiz öğrenme 
df_2.drop(['nameOrig', 'nameDest', 'isFraud'], axis=1, inplace=True)

#Denetimli öğrenme
df.drop(['nameOrig', 'nameDest'], axis=1, inplace=True) 
df.head()

In [ ]:
#Korelasyon haritasını gösterir
corr_matrix = df.corr()
plt.figure(figsize=(20,15))
sns.heatmap(corr_matrix, annot=True)
plt.show()

In [ ]:
#Denetimli öğrenme
y = df['isFraud'].values
X = df.drop('isFraud', axis=1).values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
scaler = StandardScaler() #Veriler farklı ölçeklerde olduğunda algoritmalara zorluk çıkartır. StandarScaler ile değerler 0 ile 1 arasında ölçeklenir ve bu da algoritmalar için yararlıdır
X_train = scaler.fit_transform(X_train)

#Denetimsiz öğrenme
scaler_2 = StandardScaler()
df_scaled = scaler_2.fit_transform(df_2[['step', 'type', 'amount', 'isFlaggedFraud']]) #Korelasyon matrisinde değeri yüksek çıkan 4 sütun

In [ ]:
#Yaptığım denemelerde en iyi sonucu verdiği için lojistik regresyonu seçtim. Tahmin yaptırdım ve değerlendirme metrikleriyle analiz ettim.
lr = LogisticRegression(max_iter=1000)

In [ ]:
#Denetimsiz öğrenme
parameters = {'C': [0.1, 10], 'penalty': ['l1', 'l2'], 'solver': ['liblinear']}
random_search = RandomizedSearchCV(
    estimator=lr,
    param_distributions=parameters,
    n_iter=10,
    cv=5,  # 5 katlamalı çapraz doğrulama
    n_jobs=-1,  # Tüm işlemcileri kullan
    verbose=1,
    random_state=42
)

random_search.fit(X_train, y_train)

# En iyi parametreleri ve skoru yazdıralım
print("En iyi parametreler:", random_search.best_params_)
print("En iyi doğruluk skoru:", random_search.best_score_)

# En iyi model ile tahmin
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)
lr_accuracy = accuracy_score(y_test, y_pred)
lr_precision = precision_score(y_test, y_pred)
lr_recall = recall_score(y_test, y_pred)
lr_f1 = f1_score(y_test, y_pred)

print("Logistic Regression Classifier Performance:")
print("Accuracy score: ", lr_accuracy)
print("Precision:", lr_precision)
print("Recall:", lr_recall)
print("F1 Score:", lr_f1)

lr_cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix = lr_cm, display_labels = lr.classes_)
disp.plot()
plt.show()

In [ ]:
#Denetimli öğrenme
param_grid = {
    'eps': [0.3, 0.5, 0.7],
    'min_samples': [5, 10, 15]
}

best_eps = None
best_min_samples = None
best_silhouette_score = -1  # Silhouette skoru maksimuma çıkarılacak
best_dbscan_model = None
best_labels = None

for params in ParameterGrid(param_grid):
    dbscan = DBSCAN(eps=params['eps'], min_samples=params['min_samples'])
    labels = dbscan.fit_predict(X_preprocessed)
    
    # Eğer tek bir küme varsa (ya da sadece noise varsa), silhouette score hesaplanamaz
    if len(set(labels)) > 1:
        # Silhouette skoru ile değerlendirme
        score = silhouette_score(X_preprocessed, labels)
        print(f"DBSCAN (eps={params['eps']}, min_samples={params['min_samples']}): Silhouette Score: {score}")
        
        if score > best_silhouette_score:
            best_silhouette_score = score
            best_eps = params['eps']
            best_min_samples = params['min_samples']
            best_dbscan_model = dbscan
            best_labels = labels

# Optimize edilmiş modeli kullanarak nihai kümeleme
print(f"En İyi Parametreler: eps={best_eps}, min_samples={best_min_samples}")
print(f"En İyi Silhouette Skoru: {best_silhouette_score}")

# Nihai küme etiketlerini DataFrame'e ekleme
df_2['DBSCAN_Label'] = best_labels

# Anormal (fraud) verileri belirleme (DBSCAN'de -1 anormal verileri temsil eder)
fraud_cases = df_2[df_2['DBSCAN_Label'] == -1]

print("Anormal (fraud) veriler:")
print(fraud_cases.head())

# Silhouette skoru ile değerlendirme
silhouette_avg = silhouette_score(X_preprocessed, best_labels)
print(f"Final Silhouette Score: {silhouette_avg}")

# Davies-Bouldin İndeksi ile değerlendirme (daha düşük, daha iyi demektir)
davies_bouldin_avg = davies_bouldin_score(X_preprocessed, best_labels)
print(f"Davies-Bouldin Score: {davies_bouldin_avg}")